In [32]:
from difflib import SequenceMatcher
import copy

In [29]:
a = ['a', 'bad', 'time', 'today', 'etc']
b = ['not', 'really', 'bad', 'time', 'now']

In [30]:
s = SequenceMatcher(None, a, b)

In [33]:
def pad_replacements(a_in, b_in):
    a = copy.deepcopy(a_in)
    b = copy.deepcopy(b_in)
    if len(a) > len(b):
        diff = len(a) - len(b)
        for i in range(0, diff+1):
            b.append("<eps>")
    elif len(b) > len(a):
        diff = len(b) - len(a)
        for i in range(0, diff+1):
            a.append("<eps>")
    return [x for x in zip(a, b)]    

In [27]:
pad_replacements(["a", "b"], ["a"])
pad_replacements(["a", "b", "c"], ["a"])
pad_replacements(["a"], ["a", "b", "c"])


[('a', 'a'), ('<eps>', 'b'), ('<eps>', 'c')]

In [34]:
outputs = []
for tag, i1, i2, j1, j2 in s.get_opcodes():
    print(f"{tag} {i1} {i2} {j1} {j2}")
    if tag == "equal":
        for x in a[i1:i2]:
            print(f"{tag} {i1} {i2} {j1} {j2} {x}")
            print(outputs)
            outputs.append(f"{x} {x}")
    elif tag == "insert":
        for x in b[j1:j2]:
            print(f"{tag} {i1} {i2} {j1} {j2} {x}")
            print(outputs)
            outputs.append(f"<eps> {x}")
    elif tag == "delete":
        for x in a[i1:i2]:
            print(f"{tag} {i1} {i2} {j1} {j2} {x}")
            print(outputs)
            outputs.append(f"{x} <eps>")
    elif tag == "replace":
        for x, y in pad_replacements(a[i1:i2], b[j1:j2]):
            print(f"{tag} {i1} {i2} {j1} {j2} {x}")
            print(outputs)
            outputs.append(f"{x} {y}")

replace 0 1 0 2
replace 0 1 0 2 a
[]
replace 0 1 0 2 <eps>
['a not']
equal 1 3 2 4
equal 1 3 2 4 bad
['a not', '<eps> really']
equal 1 3 2 4 time
['a not', '<eps> really', 'bad bad']
replace 3 5 4 5
replace 3 5 4 5 today
['a not', '<eps> really', 'bad bad', 'time time']
replace 3 5 4 5 etc
['a not', '<eps> really', 'bad bad', 'time time', 'today now']


In [35]:
outputs

['a not', '<eps> really', 'bad bad', 'time time', 'today now', 'etc <eps>']